In [3]:
from lxml import etree
import numpy as np
# model = ...  # your Pinocchio model

num_joints = 20 # number of joints for each arm
# The state is continuous since the arm is a flexible sheet, but we are discretizing it so 
#   that we can use the dynamics of a robot arm with n joints
num_arms = 2  # number of arms
arm_length  = 0.2  # length of each arm
mass_propeller = 0.003 # mass of each propeller
mass_base = 0.1  # mass of the base

arm_mass = 0.2 # TODO needs to be set
mass_elem = arm_mass / num_joints
point_mass_size = 0.001 # size of the point mass for each element

robot = etree.Element("robot", name="skyweave_duo")

base_link = etree.SubElement(robot, "link", name="base_link")
inertial = etree.SubElement(base_link, "inertial")
etree.SubElement(inertial, "origin", xyz="0 0 0", rpy="0 0 0")
etree.SubElement(inertial, "mass", value="1")
etree.SubElement(inertial, "inertia", ixx="1e-6", ixy="0", ixz="0", iyy="1e-6", iyz="0", izz="1e-6")
collision = etree.SubElement(base_link, "collision")
etree.SubElement(collision, "origin", xyz="0 0 0", rpy="0 0 0")
geom = etree.SubElement(collision, "geometry")
# etree.SubElement(geom, "sphere", radius=f"{3*point_mass_size}")
etree.SubElement(geom, "box", size="0.02 0.02 0.02")
vis = etree.SubElement(base_link, "visual")
etree.SubElement(vis, "origin", xyz="0 0 0", rpy="0 0 0")
geom = etree.SubElement(vis, "geometry")
etree.SubElement(geom, "box", size="0.02 0.02 0.02")
# etree.SubElement(geom, "sphere", radius=f"{30*point_mass_size}")

for arm_id in range(num_arms):
    arm_angle = arm_id * (2*np.pi / num_arms)  # angle for each arm    
    joint_disp = arm_length / num_joints  # displacement for each joint
    print(f"arm {arm_id} angle: {arm_angle}, joint displacement: {joint_disp*np.sin(arm_angle)} {joint_disp*np.cos(arm_angle)}")

    origin_pos = f"0 {joint_disp} 0"
   # Create link elements
    for i in range(num_joints):
        link = etree.SubElement(robot, "link", name=f"arm_{arm_id}_link_{i}")
        inertial = etree.SubElement(link, "inertial")
        etree.SubElement(inertial, "origin", xyz=origin_pos, rpy="0 0 0")

        # Set mass and inertia properties
        if i == num_joints - 1:  # last link has propeller 
            etree.SubElement(inertial, "mass", value=f"{mass_propeller+mass_elem}")
        else:
            etree.SubElement(inertial, "mass", value=f"{mass_elem}")
        etree.SubElement(inertial, "inertia", ixx="1e-6", ixy="0", ixz="0", iyy="1e-6", iyz="0", izz="1e-6")
        collision = etree.SubElement(link, "collision")
        etree.SubElement(collision, "origin", xyz=origin_pos, rpy="0 0 0")
        geom = etree.SubElement(collision, "geometry")
        etree.SubElement(geom, "sphere", radius=f"{point_mass_size}")
        vis = etree.SubElement(link, "visual")
        mat = etree.SubElement(vis, "material", name="red")
        etree.SubElement(mat, "color", rgba="1 0 0 1")
        etree.SubElement(vis, "origin", xyz=origin_pos, rpy="0 0 0")
        geom = etree.SubElement(vis, "geometry")
        etree.SubElement(geom, "sphere", radius=f"{15*point_mass_size}")



    arm_angle = arm_id * (2*np.pi / num_arms)  # angle for each arm    
    joint_disp = arm_length / num_joints  # displacement for each joint

    # Create joint elements
    for i in range(0, num_joints):  # skip universe
        joint_elem = None
        if i == 0:
            joint_elem = etree.SubElement(robot, "joint", name=f"arm_{arm_id}_joint_base_{i}", type="revolute")
        else:
            joint_elem = etree.SubElement(robot, "joint", name=f"arm_{arm_id}_joint_{i-1}_{i}", type="revolute")
        # etree.SubElement(joint_elem, "dynamics", damping="1e-4")
        etree.SubElement(joint_elem, "child", link=f"arm_{arm_id}_link_{i}")
        etree.SubElement(joint_elem, "axis", xyz="1 0 0")
        etree.SubElement(joint_elem, "limit", effort="1000", velocity="100", lower="-3.14", upper="3.14")
        if i == 0:
            etree.SubElement(joint_elem, "parent", link="base_link")
            # etree.SubElement(joint_elem, "origin", xyz=f"{np.round(joint_disp*np.sin(arm_angle), 6)} {np.round(joint_disp*np.cos(arm_angle), 6)} 0", rpy=f"0 0 {arm_angle}")
            etree.SubElement(joint_elem, "origin", xyz=f"0 0 0", rpy=f"0 0 {arm_angle}")
        else:
            # Only revolute X shown here — you'd need logic for other types
            etree.SubElement(joint_elem, "parent", link=f"arm_{arm_id}_link_{i-1}")
            etree.SubElement(joint_elem, "origin", xyz=origin_pos, rpy="0 0 0")

# Save to file
with open("exported.urdf", "wb") as f:
    f.write(etree.tostring(robot, pretty_print=True))


arm 0 angle: 0.0, joint displacement: 0.0 0.01
arm 1 angle: 3.141592653589793, joint displacement: 1.2246467991473532e-18 -0.01


In [10]:
import pybullet as p
import pybullet_data
import time

# Connect to PyBullet (GUI for visualization, DIRECT for no GUI)
physicsClient = p.connect(p.GUI)
# physicsClient = p.connect(p.DIRECT) # Uncomment for no GUI

# Set the search path for assets
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.resetDebugVisualizerCamera(
    cameraDistance=1.0,
    cameraYaw=75,
    cameraPitch=-20,
    cameraTargetPosition=[0, 0, 1]
)
# Load the plane (ground)
planeId = p.loadURDF("plane.urdf")


def get_link_id(body_id, target_name):
    num_joints = p.getNumJoints(body_id)
    for i in range(num_joints):
        joint_info = p.getJointInfo(body_id, i)
        link_name = joint_info[12].decode("utf-8")
        if link_name == target_name:
            return i
    return -1  # Not found

K = 1e-2 # stiffness of the joints
D = 1e-4 # damping of the joints

# Define the initial position and orientation of the robot
startPos = [0, 0, 1]
startOrientation = p.getQuaternionFromEuler([0, 0, 0])

# Load the robot URDF
robotId = p.loadURDF("exported.urdf", startPos, startOrientation, useFixedBase=True)

# Optionally, set additional parameters like gravity
p.setGravity(0, 0, -9.81)

print(f"the joitn state is {p.getJointState(robotId, 0)}")

# Simulation loop
for i in range(1000):
    for id in range(21):
        # Set joint positions (for example, to zero)
        if id < 5:
            K = 2e-2
        elif id < 10:
            K = 1e-2
        elif id < 15:
            K = 5e-3
        else:
            K = 3e-4
 
        p.setJointMotorControl2(
            bodyIndex=robotId,
            jointIndex=id,
            controlMode=p.TORQUE_CONTROL,
            # higher stiffness for smaller id
           force=K*(0 - p.getJointState(robotId, id)[0]) 
            + D*(0 - p.getJointState(robotId, id)[1]),  # Set to desired position
        )
    if True:
        p.applyExternalForce(
        objectUniqueId=robotId,
        linkIndex=get_link_id(robotId, "arm_0_link_19"),
        forceObj=[0, 0, 0.008],
        posObj=[0, 0, 0],  # Where the force is applied
        flags=p.LINK_FRAME
        )

    p.stepSimulation()
    time.sleep(1./240.)


p.disconnect()


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 3060 Laptop GPU/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 545.29.06
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 545.29.06
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 3060 Laptop GPU/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation
ven = NVIDIA Corporation
the joitn state is (0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0)
numActiveThreads = 0
stopping threads
